## PART B

### Student name : Bhagyashree Manoj Keswani
### Student ID: 1006790849

In [0]:
#all spark imports
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col

from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator
#instantiate the spark session
spark = SparkSession.builder.appName("assignment2_partB").getOrCreate()

### QN 1) Describe your data. Calculate top 15 movies with highest ratings and top 10 users who provided highest ratings.

##### Describing the data:

In [0]:
path="/FileStore/shared_uploads/bhagyashree.keswani@mail.utoronto.ca/movies.csv"
df = spark.read.format("csv")\
.option("header",True)\
.option("inferSchema", True)\
.option("sep",',')\
.option("path",path)\
.load()      
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|                 1|                 0|
|    max|                99|                 5|                29|
+-------+------------------+------------------+------------------+



##### There are no null values in the dataset. The data consists of 100 movies rated between 1-5 by 30 different users. The average rating given is 1.77 with a standard deviation of 1.18.

In [0]:
#Top 15 highest rated movies
df.groupBy('movieID').agg(count('rating')\
.alias('count_of_ratings')).orderBy(desc('count_of_ratings'))\
.show(15)

+-------+----------------+
|movieID|count_of_ratings|
+-------+----------------+
|      6|              20|
|     29|              20|
|     51|              20|
|     22|              20|
|     50|              20|
|     94|              19|
|     55|              19|
|     68|              19|
|      2|              19|
|     15|              19|
|     85|              18|
|     36|              18|
|     86|              18|
|     88|              18|
|     45|              18|
+-------+----------------+
only showing top 15 rows



In [0]:
#Top 10 users who provided highest ratings
df.groupBy('userID').count().sort(col('count').desc()).show(10)

+------+-----+
|userID|count|
+------+-----+
|    14|   57|
|     6|   57|
|    22|   56|
|    11|   56|
|     4|   55|
|    12|   55|
|     7|   54|
|     9|   53|
|    18|   52|
|    23|   52|
+------+-----+
only showing top 10 rows



### QN 2) Split dataset into train and test. Try 2 different combinations for e.g. (60/40, 70/30, 75/25 and 80/20). (Train your model and use collaborative filtering approach).

In [0]:
#Define ALS
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",coldStartStrategy="drop")

#Splitting into train(70% and 60%) and test(30% and 40%):
train1, test1 = df.randomSplit([0.7,0.3], seed=1628)
train2, test2 = df.randomSplit([0.6,0.4], seed=1628)

In [0]:
#Fit the first model on training data
model1 = als.fit(train1)

#Get the predictions on the first test set
prediction1=model1.transform(test1)

print('MODEL 1 Predictions are')
prediction1.show()

MODEL 1 Predictions are
+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     27|     1|     0| 1.5529307|
|     30|     1|     0| 1.4506685|
|     34|     1|     0|0.91748166|
|     41|     2|     0| 1.0085716|
|     48|     1|     0|0.76005566|
|     51|     1|     0| 1.3591791|
|     55|     1|     0|  0.749434|
|     67|     1|     0| 1.2915943|
|     68|     1|     0| 1.8293437|
|     72|     1|     0| 1.3652861|
|     79|     1|     0|0.79434454|
|     87|     1|     0| 0.7555055|
|     92|     4|     0| 2.7689922|
|     94|     1|     0|  2.166491|
|      9|     3|     1|0.80412495|
|     14|     1|     1| 0.9059999|
|     36|     2|     1| 1.5087504|
|     40|     1|     1|  1.292395|
|     41|     2|     1|  1.402406|
|     44|     1|     1| 1.1161789|
+-------+------+------+----------+
only showing top 20 rows



In [0]:
#Fit the second model on training data
model2= als.fit(train2)

#Get the predictions on the second test set
prediction2=model2.transform(test2)

print('MODEL 2 predictions are')
prediction2.show()

MODEL 2 predictions are
+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      3|     1|     0| 1.3519063|
|     26|     3|     0| 1.3455426|
|     27|     1|     0|  1.112159|
|     30|     1|     0| 1.6367576|
|     34|     1|     0| 1.0226688|
|     37|     1|     0| 1.2747121|
|     41|     2|     0| 0.7807222|
|     46|     1|     0|  2.166997|
|     48|     1|     0| 0.4200258|
|     51|     1|     0|  1.184576|
|     55|     1|     0| 1.1124299|
|     67|     1|     0| 1.0818979|
|     68|     1|     0| 1.6705896|
|     69|     1|     0|  0.426923|
|     72|     1|     0| 1.4460303|
|     77|     2|     0| 1.0396996|
|     79|     1|     0| 0.8116931|
|     87|     1|     0|  1.226676|
|     92|     4|     0| 2.8300445|
|     94|     1|     0| 2.2993355|
+-------+------+------+----------+
only showing top 20 rows



##### The two models are fit on the two sets of train data and the two predictions are displayed above. In the next part, the errors will be calculated.

### QN 3) Explain MSE, RMSE and MAE. Compare and evaluate both of your models with evaluation metrics (RMSE or MAE). Describe which one works better and why?

#### Mean Squared Error(MSE):
Mean squared error is a measure of the average sqaured difference between actual and predicted values in the dataset. For every data point, the distance from every point to the corresponding y value on the curve fit is computed and the distance is then squared. Then, the values of distance for all data points are added to get the error. The smaller the Mean Squared Error, the closer the fit is to the data. It is also known as mean squared deviation. 

#### Root Mean Squared Error(RMSE):
Root mean squared error (RMSE) is the square root of the mean squared error in simple terms. MSE is highly biased for higher values. RMSE has the benefit of penalizing large errors more so is more appropriate in some cases. RMSE has better performance when dealing with large error values.  RMSE is more widely used than MSE as it has the same units as the dependent variable in the regression model. 

#### Mean Absolute Error(MAE):
MAE is a measure of absolute difference between predicted and original values in the data. It computes the average of residuals in data. It doesn't account for positive/negative values and may not adequately reflect the performance when dealing with large error values.

In [0]:
rmse_evaluator = RegressionEvaluator(metricName="rmse",\
                                     labelCol="rating",predictionCol="prediction")
mse_evaluator = RegressionEvaluator(metricName="mse",\
                                    labelCol="rating",predictionCol="prediction")
mae_evaluator = RegressionEvaluator(metricName="mae",\
                                    labelCol="rating",predictionCol="prediction")

rmse1 = rmse_evaluator.evaluate(prediction1)
rmse2 = rmse_evaluator.evaluate(prediction2)

mse1=mse_evaluator.evaluate(prediction1)
mse2=mse_evaluator.evaluate(prediction2)

mae1=mae_evaluator.evaluate(prediction1)
mae2=mae_evaluator.evaluate(prediction2)

print('**RMSE**')
print("Root-mean-square error for 1st model = " + str(rmse1))
print("Root-mean-square error for 2nd model = " + str(rmse2))

print('\n**MSE**')
print("Mean-squared error for 1st model = " + str(mse1))
print("Mean-squared error for 2nd model = " + str(mse2))

print('\n**MAE**')
print("Mean-absolute error for 1st model = " + str(mae1))
print("Mean-absolute error for 2nd model = " + str(mae2))

**RMSE**
Root-mean-square error for 1st model = 1.0873905616065613
Root-mean-square error for 2nd model = 1.1483008947559354

**MSE**
Mean-squared error for 1st model = 1.1824182334710327
Mean-squared error for 2nd model = 1.3185949448972818

**MAE**
Mean-absolute error for 1st model = 0.7619639384899897
Mean-absolute error for 2nd model = 0.7898707925069988


##### Observations: The results above indicate that model 1(with 7:3 train-test ratio) is better as it has lower MAE, MSE and RMSE. This implies that the distance between average and actual ratings is lower for Model 1.  The 1st model performace is very good when it comes to RMSE as its RMSE is just 1 when the data size is 1000+. Model 1 has RMSE 0.06 lower than Model 2, an MSE 0.13 lower than Model 2 and its MAE is 0.01 lower than Model 2. The differences are not significantly large but we conclude that a split of 7:3 is better and will use the same split to tune the model in the next part.

### QN 4) Now tune the parameters of your algorithm to get the best set of parameters. Explain different parameters of the algorithm which you have used for tuning your algorithm. Evaluate all your models again.

In [0]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator,TrainValidationSplit
param_grid = (ParamGridBuilder()\
              .addGrid(als.rank, [8,10])\
              .addGrid(als.regParam, [.3, .5, 1])\
              .addGrid(als.maxIter,[10,15])\
              .addGrid(als.alpha,[0.5,1.0])\
              .build())

tvs=TrainValidationSplit(estimator=als, estimatorParamMaps=param_grid,\
                         evaluator=rmse_evaluator, trainRatio=0.8)
tuned_model=tvs.fit(train1)

In [0]:
best_model = tuned_model.bestModel
test_pred1=best_model.transform(test1)
tuned_rmse=rmse_evaluator.evaluate(test_pred1)
tuned_mse=mse_evaluator.evaluate(test_pred1)
tuned_mae=mae_evaluator.evaluate(test_pred1)
print('The RMSE of the tuned model is',tuned_rmse)
print('The MSE of the tuned model is',tuned_mse)
print('The MAE of the tuned model is',tuned_mae)

best_rank=best_model._java_obj.parent().getRank()
best_regParam=best_model._java_obj.parent().getRegParam()
best_maxIter=best_model._java_obj.parent().getMaxIter()
best_alpha=best_model._java_obj.parent().getAlpha()

print('The best hyperparameters for the model are \nRank:',best_rank,\
      '\nregParam:',best_regParam,\
      '\nmaxIter:',best_maxIter,\
      '\nAlpha:',best_alpha)

The RMSE of the tuned model is 1.1137152572960776
The MSE of the tuned model is 1.2403616743340682
The MAE of the tuned model is 0.7519136629248625
The best hyperparameters for the model are 
Rank: 8 
regParam: 0.3 
maxIter: 15 
Alpha: 0.5


##### Explaining different parameters used for tuning the algorithm:
- The first hyperparameter tuned is the rank. Rank is the number of latent factors in the model. It controls the number of internal parameters that must be fit from the data. Too many can lead to overfitting. However, as the size of data increases, increasing rank can improve accuracy. Here, we tune with ranks of 8 and 10(default=10)

- The second one  is Regularization parameter which helps tackle overfitting in the model. Here, we tune with values of 0.3,0.5 and 1. Too small values can lead to overfitting while large values cause the data to underfit.(default=1)

- Third is Maximum iterations which determines the time required for model to converge. Here, we tune with 10 and 15 iterations.(default=10)

- Fourth is alpha alpha which is applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations. Here we tune it with 0.5 and 1. (default=1)

#### Inference from best tuned model :
Except for MAE, RMSE and MSE are higher for the tuned model as compared to the default model fitted in the previous part. This partially explains why tuning does not always help improve model performance and can sometimes lead to overfitting.

- Nevertheless, the best rank is found to be 8, lower than the default value 10. This means that model is better fit with fewer internal parameters.

- Best regparam is 0.3, lower than 1.0, which might be the reason the model maybe slightly overfitting.

- Best maxIter is 15, higher than default of 10.

- Best alpha is 0.5, lower than default of 1.

### QN 5) Calculate top 10 movies recommendations for user id 9 and user id 13.

In [0]:
#Define a function to recommend top 10 movies to a particular user 
def movie_recommendations(user):
    #filter the data of user with the specific id 
    userdf=df.filter(df.userId.isin([user])).sort('rating',ascending=False)
    
    #recommend 50 movies to this user
    users_recommendn=best_model.recommendForUserSubset(userdf,50)
    
    #As the output obtained above is in the form of a dictionary
    #The column recommendations is exploded to get the movie ID and sorted rating 
    users_recommendn2 = users_recommendn.select('userId',explode('recommendations').\
                                                alias('Recommendations'))\
    .select('userId','Recommendations.*')\
    .sort('rating',ascending=False)

    users_recommendn2.select('movieID','userId','rating')
    
    #now out of these 50 movies, some have already been watched by the user
    #hence, we exclude these movies from the recommendations by doing a left_anti join.
    userid_recommendn=users_recommendn2.join(userdf,on=['movieID'],how='left_anti')
    
    #show the top 10 movie recommendations for the user
    userid_recommendn.show(10)

In [0]:
#Getting the top 10 movie recommendations for user ID 9
movie_recommendations(9)

+-------+------+---------+
|movieId|userId|   rating|
+-------+------+---------+
|     23|     9| 2.432024|
|     62|     9|2.2736871|
|     92|     9|2.1884024|
|     18|     9|2.1234174|
|     46|     9|2.0931466|
|     29|     9|2.0540233|
|     74|     9|1.9724503|
|     13|     9|1.9195827|
|     55|     9|1.9127926|
|     96|     9|1.8557006|
+-------+------+---------+
only showing top 10 rows



In [0]:
#Getting the top 10 movie recommendations for user ID 13
movie_recommendations(13)

+-------+------+---------+
|movieId|userId|   rating|
+-------+------+---------+
|     30|    13|2.1282873|
|      2|    13|1.9215084|
|     92|    13|1.6789302|
|     96|    13|1.6434557|
|     69|    13| 1.586558|
|     75|    13|1.5734578|
|     39|    13|1.5606692|
|     32|    13|1.5472282|
|     58|    13|1.4019821|
|     70|    13|1.3428351|
+-------+------+---------+
only showing top 10 rows

